In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd "/content/drive/My Drive/Colab Notebooks/slGan"

In [0]:
%pwd

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [0]:
# mnistを使ってGANを実行してみる
# https://www.nogawanogawa.com/entry/2018/03/17/181456

from tensorflow.examples.tutorials.mnist import input_data

In [0]:
# 縦横４枚ずつ画像を描画
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [0]:
#https://github.com/dylanthomas/tensorflow/blob/master/xavier_init.py
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                             minval=low, maxval=high,
                             dtype=tf.float32)

In [0]:
# generatorの入力となる乱数
Z = tf.placeholder(tf.float32, shape=[None, 100])

In [0]:
#　generatorのパラメータの初期化
G_W1 = tf.Variable(xavier_init(100, 128))
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init(128, 784))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

In [77]:
print(G_W1)
print(G_b1)

<tf.Variable 'Variable_16:0' shape=(100, 128) dtype=float32_ref>
<tf.Variable 'Variable_17:0' shape=(128,) dtype=float32_ref>


In [0]:
theta_G = [G_W1, G_W2, G_b1, G_b2]

In [54]:
print(theta_G)

[<tf.Variable 'Variable_16:0' shape=(100, 128) dtype=float32_ref>, <tf.Variable 'Variable_18:0' shape=(128, 784) dtype=float32_ref>, <tf.Variable 'Variable_17:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'Variable_19:0' shape=(784,) dtype=float32_ref>]


In [0]:
#　discriminator用の教師データのplaceholder [28x28=784]
X = tf.placeholder(tf.float32, shape=[None, 784])

In [0]:
# discriminatorのパラメータの初期化
D_W1 = tf.Variable(xavier_init(784, 128))
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init(128, 1))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

In [0]:
theta_D = [D_W1, D_W2, D_b1, D_b2]

In [0]:
# generatorのセットアップ
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [0]:
# discriminatorのセットアップ
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit

In [0]:
# Flowのセットアップ
G_sample = generator(Z) #type float32 

In [0]:
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

In [0]:
# Lossの計算のセットアップ
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [0]:
# discriminatorのパラメータの更新
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

In [0]:
# generatorのパラメータの更新
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [0]:
# iteration 1000回毎にサンプル画像を生成するときのgeneratorの入力を生成
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [0]:
batch_size = 128
Z_dim = 100

In [0]:
# sessionの定義
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [73]:
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [0]:
# 画像生成用ディレクトリの作成
if not os.path.exists('output/'):
    os.makedirs('output/')

In [0]:
#　画像ファイルの通番初期化
i = 0

# 学習プロセス開始
for itr in range(10000):
    if itr % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('output/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(batch_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})

    if itr % 1000 == 0:
        print('Iter: {}'.format(itr))
        print('D_loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()